# Библиотеки и гиперпараметры

In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchdiffeq
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.optimize import minimize
from torch.utils.data import DataLoader, TensorDataset
import os
import random
from torchsummary import summary

#from scipy.ndimage import minimum_filter


# Параметры
is_restart = True
p_cutoff = 0.01
n_epoch = 1000
n_plot =100
datasize = 100
tstep = 0.4
n_exp_train = 20
n_exp_test = 10
n_exp = n_exp_train + n_exp_test
noise = 5.e-2
ns = 5
nr = 4
k = torch.tensor([0.1, 0.2, 0.13, 0.3], dtype=torch.float32)                                                              
atol = 1e-5
rtol = 1e-2
maxiters = 10000
lb = 1e-5
ub = 10.0
b0 = -10.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Система ОДУ

In [82]:
# Функция ODE

def true_ode_func(t, y, k):
    dydt = torch.zeros_like(y)
    dydt[0] = -2 * k[0] * y[0]**2 - k[1] * y[0]
    dydt[1] = k[0] * y[0]**2 - k[3] * y[1] * y[3]                                                                        
    dydt[2] = k[1] * y[0] - k[2] * y[2]
    dydt[3] = k[2] * y[2] - k[3] * y[1] * y[3]
    dydt[4] = k[3] * y[1] * y[3]
    return dydt



# Генерация начальных и истинных данных

In [85]:

# Генерация данных

np.random.seed()
torch.manual_seed(123)
u0_list = np.random.rand(n_exp, ns).astype(np.float32)
u0_list[:, :2] += 0.2
u0_list[:, 2:] = 0.0    # u0_list - матрица (30x5) нач.значений концентраций для 30 экспериментов в диапазоне: [0.2 0.2 0 0 0] - [1.2 1.2 0 0 0]

#print("u0_list: ", u0_list) 
tsteps = torch.linspace(0, datasize * tstep, datasize, dtype=torch.float32) #временной промежуток от 0 до 40 по 100 точек через 0.4
#print(tsteps)
ode_data_list = []
print(ode_data_list)



std_list = []

for i in range(n_exp):   #   Вычисление точных изменений концентраций со временем
    u0 = torch.tensor(u0_list[i], dtype=torch.float32)
    #print("u0 torch tensor ", i, ": ",u0)
    
    # Решение системы ОДУ с использованием torchdiffeq
    sol = torchdiffeq.odeint(lambda t, y: true_ode_func(t, y, k), u0, tsteps, atol=atol, rtol=rtol, method='dopri5')                                                  
    #print(sol.shape)
    
    # Добавление шума к данным
    noise_tensor = torch.randn_like(sol) * sol * noise
    ode_data = sol + noise_tensor
    
    # Сохранение данных
    ode_data_list.append(ode_data)
    
    # Вычисление стандартного отклонения
    std_list.append(torch.max(ode_data, dim=0).values - torch.min(ode_data, dim=0).values + lb)
    
ode_data_list = torch.stack(ode_data_list)  #(30, 100, 5) изменения концентраций 5 веществ на 100 временных точках для каждого из 30 экспериментов

##########
#TESTING
##########
# print(ode_data_list[1])

# testing = ode_data_list[1].numpy()
# np.set_printoptions(
#     precision=8,   # Количество знаков после запятой
#     suppress=True  # Отключаем научную нотацию
# )
# print(testing)


# Объединение результатов и вычисление максимальных значений
y_std = torch.max(torch.stack(std_list), dim=0).values
print(y_std)



[]
[[0.30060828 0.4959544  0.         0.         0.        ]
 [0.25669804 0.46596438 0.02297995 0.0006183  0.00001122]
 [0.23974599 0.4760674  0.03883144 0.00213248 0.00008543]
 [0.21247564 0.54100037 0.05636562 0.0045682  0.00028803]
 [0.17413263 0.433385   0.06804284 0.00812639 0.00061753]
 [0.17224872 0.44045922 0.08126694 0.01091905 0.00114399]
 [0.15825462 0.47682175 0.09361083 0.01494218 0.00185241]
 [0.14083852 0.47825405 0.10202014 0.01916588 0.00296744]
 [0.12675728 0.4412838  0.09941486 0.02268582 0.00453115]
 [0.11639027 0.4731546  0.12343784 0.02595499 0.00554385]
 [0.1219348  0.40945145 0.11678772 0.02964299 0.00711199]
 [0.09409709 0.4510641  0.11911025 0.03930083 0.00800027]
 [0.08766151 0.4950046  0.11908669 0.04239975 0.01129204]
 [0.07856152 0.44639793 0.11930326 0.04420518 0.01257138]
 [0.07916946 0.42137256 0.12071745 0.04793482 0.01534354]
 [0.07421731 0.46749088 0.12269989 0.05068589 0.01977741]
 [0.07241191 0.49251956 0.11722346 0.05626111 0.02163228]
 [0.0601458

# CRNN Model

In [88]:
# Определение модели CRNN
class CRNN(nn.Module):
    def __init__(self):
        super(CRNN, self).__init__()
        self.w_b = nn.Parameter(torch.randn(nr) + b0)
        w_out_clamped  = torch.clamp(torch.randn(ns, nr),-2.5, 2.5)
        w_out_clamped[torch.abs(w_out_clamped) < p_cutoff] = 0.0
        self.w_out = nn.Parameter(w_out_clamped)

        print("w_in: ",  torch.clamp(-self.w_out, 0, 2.5).T)
    
    def forward(self, t, u):
        
        w_in = torch.clamp(-self.w_out, 0, 2.5)
        
        w_in_x = torch.matmul(w_in.T, torch.log(torch.clamp(u, lb, ub)))                                                         
        du = torch.matmul(self.w_out, torch.exp(w_in_x + self.w_b))

        return du

    # def printInfo(self):

    #     print("Bias w_b: \n     ", self.w_b )
    #     print("Output w_out: \n     ", self.w_out)

    #     print("Grad of Bias : \n     ",  self.w_b.grad)
    #     print("Grad of Output : \n     ",  self.w_out.grad)
        
##########
#TESTING
##########

# model = CRNN()
# print(model.w_out.data)
# print(f"Model structure: {model}\n")
# gen_p = model.parameters() # возвращает генератор с набором параметров
# print(list(model.parameters())) # отображение списка параметров

#model.printInfo()


w_in:  tensor([[0.6600, 0.0000, 0.0000, 0.4709, 1.4033],
        [0.0000, 1.9188, 0.0000, 0.0000, 0.6057],
        [0.8968, 1.4377, 0.0000, 0.1935, 0.0000],
        [1.7302, 2.5000, 1.5946, 1.1369, 0.0000]], grad_fn=<PermuteBackward0>)
tensor([[-0.6600,  0.4327, -0.8968, -1.7302],
        [ 0.1499, -1.9188, -1.4377, -2.5000],
        [ 0.0712,  0.1655,  0.0290, -1.5946],
        [-0.4709,  0.6804, -0.1935, -1.1369],
        [-1.4033, -0.6057,  0.5546,  0.3441]])
Model structure: CRNN()

[Parameter containing:
tensor([-10.0202,  -9.1111,  -9.6397, -10.8175], requires_grad=True), Parameter containing:
tensor([[-0.6600,  0.4327, -0.8968, -1.7302],
        [ 0.1499, -1.9188, -1.4377, -2.5000],
        [ 0.0712,  0.1655,  0.0290, -1.5946],
        [-0.4709,  0.6804, -0.1935, -1.1369],
        [-1.4033, -0.6057,  0.5546,  0.3441]], requires_grad=True)]


# Предсказание

In [90]:
def predict_neuralode(Model, u0):
    
    solution = torchdiffeq.odeint(func=Model, y0=u0, t=tsteps, method='dopri5', atol=1e-5, rtol=1e-2, options={'max_num_steps': 10000})
    solution = torch.clamp(solution, -10.0, 10.0)
    return solution

##########
#TESTING
##########

# model = CRNN()
# print(model.w_b.data.clone())
# print(model.w_out.data.clone())
# w_b_test = model.w_b.data.clone()
# w_out_test = model.w_out.data.clone()

# u0 = torch.tensor(u0_list[1], dtype=torch.float32)
# print("u0", u0)
# y = model(0.0, u0)
# pred=predict_neuralode(model, u0)
# print("y: ", y)
# print(pred)
# mae = torch.mean(torch.abs(pred - ode_data_list[1]))
# print(mae)

w_in:  tensor([[0.2779, 0.2422, 0.0000, 0.6124, 1.0513],
        [0.0000, 0.6172, 0.0000, 0.0000, 1.5539],
        [1.6458, 0.0000, 0.6416, 0.0000, 0.0000],
        [1.5123, 0.0000, 1.1132, 0.0000, 0.0000]], grad_fn=<PermuteBackward0>)
tensor([-10.6305, -11.4045, -10.0712,  -8.9618])
tensor([[-0.2779,  1.2409, -1.6458, -1.5123],
        [-0.2422, -0.6172,  0.3169,  0.4598],
        [ 0.5910,  0.2146, -0.6416, -1.1132],
        [-0.6124,  1.9505,  0.3941,  0.4645],
        [-1.0513, -1.5539,  0.4710,  1.7062]])
u0 tensor([0.2972, 0.4586, 0.0000, 0.0000, 0.0000])
y:  tensor([-5.9360e-09,  1.1522e-09, -2.3430e-09,  1.4271e-09,  1.7692e-09],
       grad_fn=<MvBackward0>)
tensor([[ 2.9715e-01,  4.5856e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 2.9715e-01,  4.5856e-01, -9.4667e-10,  5.7663e-10,  7.1482e-10],
        [ 2.9715e-01,  4.5856e-01, -1.8933e-09,  1.1533e-09,  1.4296e-09],
        [ 2.9715e-01,  4.5856e-01, -2.8400e-09,  1.7299e-09,  2.1445e-09],
        [ 2.9715e-01,  4

# Функция потерь

In [92]:
def loss_function(predict, target):
    mae = torch.mean(torch.abs(predict - target)/y_std)
    return mae



# Отображение параметров

In [95]:
def display_params(w_b, w_out):
    
    print("species (column) reaction (row)")
    w_in = torch.clamp(-w_out, 0, 2.5)
    
    print("\nw_in")
    print(torch.round(w_in.mT * 1000) / 1000)  # Округление до 3 знаков через масштабирование
    
    print("\nw_b")
    exp_w_b = torch.exp(w_b)
    print(torch.round(exp_w_b * 1000) / 1000)
    
    print("\nw_out")
    print(torch.round(w_out.mT * 1000) / 1000)
    
    print("\n\n####_display_params worked_####")

display_params(w_b_test, w_out_test)

species (column) reaction (row)

w_in
tensor([[0.2780, 0.2420, 0.0000, 0.6120, 1.0510],
        [0.0000, 0.6170, 0.0000, 0.0000, 1.5540],
        [1.6460, 0.0000, 0.6420, 0.0000, 0.0000],
        [1.5120, 0.0000, 1.1130, 0.0000, 0.0000]])

w_b
tensor([0., 0., 0., 0.])

w_out
tensor([[-0.2780, -0.2420,  0.5910, -0.6120, -1.0510],
        [ 1.2410, -0.6170,  0.2150,  1.9500, -1.5540],
        [-1.6460,  0.3170, -0.6420,  0.3940,  0.4710],
        [-1.5120,  0.4600, -1.1130,  0.4650,  1.7060]])


####_display_params worked_####


# Графики концентраций

In [97]:
def cbi(predict, i_exp):

    os.makedirs('figs_experiments', exist_ok=True)
    species = ["A", "B", "C", "D", "E"]
    ode_data = ode_data_list[i_exp].T  # (ns, datasize)
    #print(ode_data.T)
    predicted = predict.T.detach().numpy()
    #print(predicted.shape)
    
    fig, axes = plt.subplots(nrows=ns, figsize=(6, 12), sharex=True)
    for i in range(ns):
        ax = axes[i]
        ax.scatter(tsteps.numpy(), ode_data[i].numpy(), label="Exp", facecolors='none', edgecolors='black')
        ax.plot(tsteps.numpy(), predicted[i], label="CRNN", color='orange')
        ax.set_ylabel(f"[{species[i]}]")
        if i == 0:
            ax.legend(frameon=False)
    
    axes[-1].set_xlabel("Time")
    plt.tight_layout()
    plt.savefig(f"figs_experiments/i_exp_{i_exp}.png")
    plt.close()
    #plt.show()
    return False

# Вызов функции для i-го эксперимента
cbi(pred, 1)

False

# Графики функции потерь

In [100]:
list_loss_train = []
list_loss_val = []
iter_count = 1
def cb(Model, loss_train, loss_val):
    global list_loss_train, list_loss_val, iter_count


    list_loss_train.append(loss_train)
    list_loss_val.append(loss_val)

    w_b_disp = Model.w_b.data.clone()
    w_out_disp = Model.w_out.data.clone()

    if iter_count % n_plot == 0:
        display_params(w_b_disp, w_out_disp)

        print(f"min loss train: {min(list_loss_train):.4e}, val: {min(list_loss_val):.4e}")

        list_exp = [random.randint(0, n_exp - 1)]
        print("list_exp: ",list_exp)
        print(f"update plot for {list_exp}")

        for i_exp in list_exp:
            print("\n updated plot_",i_exp)
            u0 = torch.tensor(u0_list[i_exp], dtype=torch.float32)
            predict = predict_neuralode(Model, u0)
            cbi(predict, i_exp)

        # График лосса
        plt.figure()
        plt.plot(list_loss_train, label="train")
        plt.plot(list_loss_val, label="val")
        plt.xscale("log")
        plt.yscale("log")
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        plt.legend()
        plt.title("Training & Validation Loss")
        plt.grid(True)
        plt.savefig(f"figs_experiments/loss_plot.png")
        #plt.show()
        plt.close()

    iter_count += 1

# Обучение модели

In [103]:
model = CRNN()#.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
loss_epoch = torch.zeros(n_exp, dtype=torch.float32)  # для логов
epochs = tqdm(range(n_epoch))

for epoch in epochs:

    # Перемешивание порядка экспериментов для стахастичности
    i_exp_list = torch.randperm(n_exp_train)

    for i_exp in i_exp_list:
        optimizer.zero_grad()

        # Получаем начальное состояние и целевые данные
        u0 = torch.tensor(u0_list[i_exp], dtype=torch.float32)
        target = ode_data_list[i_exp]

        # Получаем предсказание через forward
        pred = predict_neuralode(model, u0)  

        # Считаем loss
        loss = torch.mean(torch.abs(pred - target) / y_std)

        # Обратное распространение ошибки и шаг оптимизации
        loss.backward()
        optimizer.step()
        #model.printInfo()


        model.w_out.data.clamp_(-2.5, 2.5)
        model.w_out.data[torch.abs(model.w_out.data) < p_cutoff] = 0.0

    ######## Validation ########

    with torch.no_grad():  # блок без градиентов
        for i_exp in range(n_exp):
            u0 = torch.tensor(u0_list[i_exp], dtype=torch.float32)
            target = ode_data_list[i_exp]

            pred = predict_neuralode(model, u0)  # forward
            loss_epoch[i_exp] = torch.mean(torch.abs(pred - target) / y_std)

    loss_train = torch.mean(loss_epoch[:n_exp_train]).item()
    loss_val = torch.mean(loss_epoch[n_exp_train:]).item()

    # Вывод в tqdm
    epochs.set_description(f"Loss train {loss_train:.4e} val {loss_val:.4e}")

    # Сохранение логов или callback
    cb(model, loss_train, loss_val)


w_in:  tensor([[0.0417, 0.0000, 0.1511, 0.0924, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.6895, 0.0000],
        [0.0000, 0.8158, 0.0625, 0.0000, 0.0000],
        [0.0000, 0.2403, 0.0000, 0.0000, 0.4110]], grad_fn=<PermuteBackward0>)


Loss train 5.2366e-02 val 5.3659e-02:  10%|▉         | 99/1000 [03:29<38:22,  2.56s/it]

species (column) reaction (row)

w_in
tensor([[0.9190, 0.4540, 0.0000, 0.0490, 0.0590],
        [-0.0000, 0.4550, 0.5210, 0.4100, 0.0000],
        [0.3060, 0.0000, 0.4360, 0.0000, 0.0200],
        [1.7440, 0.0000, 0.0000, 0.0000, 0.1180]])

w_b
tensor([0.2070, 0.2410, 0.0320, 0.2670])

w_out
tensor([[-0.9190, -0.4540,  1.0720, -0.0490, -0.0590],
        [ 0.0000, -0.4550, -0.5210, -0.4100,  0.5310],
        [-0.3060,  0.0930, -0.4360,  2.2070, -0.0200],
        [-1.7440,  0.9880,  0.1530,  0.5710, -0.1180]])


####_display_params worked_####
min loss train: 3.6484e-02, val: 3.4837e-02
list_exp:  [2]
update plot for [2]

 updated plot_ 2


Loss train 1.7245e-02 val 1.5714e-02:  20%|█▉        | 199/1000 [07:39<33:59,  2.55s/it]

species (column) reaction (row)

w_in
tensor([[0.9510, 0.1250, 0.0000, -0.0000, 0.0000],
        [-0.0000, 0.8080, 0.1250, 0.8310, 0.0000],
        [-0.0000, 0.0000, 1.0000, 0.0000, -0.0000],
        [1.8480, 0.0000, 0.0000, -0.0000, 0.1320]])

w_b
tensor([0.1710, 0.3140, 0.0970, 0.3560])

w_out
tensor([[-0.9510, -0.1250,  0.9990,  0.0000,  0.0890],
        [ 0.0000, -0.8080, -0.1250, -0.8310,  0.6640],
        [ 0.0000,  0.2030, -1.0000,  1.4790,  0.0000],
        [-1.8480,  0.7660,  0.2630,  0.0000, -0.1320]])


####_display_params worked_####
min loss train: 1.6908e-02, val: 1.5406e-02
list_exp:  [16]
update plot for [16]

 updated plot_ 16


...

Loss train 1.2984e-02 val 1.2502e-02:  90%|████████▉ | 899/1000 [34:36<03:47,  2.25s/it]

species (column) reaction (row)

w_in
tensor([[0.8640, -0.0000, 0.0000, -0.0000, -0.0000],
        [-0.0000, 0.9980, -0.0000, 1.0020, 0.0000],
        [-0.0000, -0.0000, 0.9930, 0.0000, -0.0000],
        [2.0250, 0.0000, 0.0000, -0.0000, -0.0000]])

w_b
tensor([0.1730, 0.3030, 0.1270, 0.1350])

w_out
tensor([[-0.8640,  0.0000,  0.8490,  0.0000,  0.0000],
        [ 0.0000, -0.9980,  0.0000, -1.0020,  0.9980],
        [ 0.0000,  0.0000, -0.9930,  1.0110,  0.0000],
        [-2.0250,  0.7680,  0.4960,  0.0000,  0.0000]])


####_display_params worked_####
min loss train: 1.2875e-02, val: 1.2480e-02
list_exp:  [0]
update plot for [0]

 updated plot_ 0


Loss train 1.2938e-02 val 1.2572e-02: 100%|█████████▉| 999/1000 [38:17<00:02,  2.23s/it]

species (column) reaction (row)

w_in
tensor([[0.8640, -0.0000, 0.0000, -0.0000, -0.0000],
        [-0.0000, 1.0030, -0.0000, 1.0030, 0.0000],
        [-0.0000, -0.0000, 0.9950, 0.0000, -0.0000],
        [2.0350, 0.0000, 0.0000, -0.0000, -0.0000]])

w_b
tensor([0.1730, 0.3020, 0.1280, 0.1360])

w_out
tensor([[-0.8640,  0.0000,  0.8490,  0.0000,  0.0000],
        [ 0.0000, -1.0030,  0.0000, -1.0030,  0.9970],
        [ 0.0000,  0.0000, -0.9950,  1.0120,  0.0000],
        [-2.0350,  0.7600,  0.4950,  0.0000,  0.0000]])


####_display_params worked_####
min loss train: 1.2875e-02, val: 1.2480e-02
list_exp:  [10]
update plot for [10]

 updated plot_ 10


Loss train 1.2938e-02 val 1.2572e-02: 100%|██████████| 1000/1000 [38:18<00:00,  2.30s/it]
